In [3]:
import glob
import pandas as pd
cat_df = pd.DataFrame()
cat_df["path"] = glob.glob("train/cat.*.*")
cat_df["target"] = 0
dog_df = pd.DataFrame()
dog_df["path"] = glob.glob("train/dog.*.*")
dog_df["target"] = 1
pd.concat([cat_df, dog_df], axis=0)

,path,target
0,train/cat.5077.jpg,0
1,train/cat.2718.jpg,0
2,train/cat.10151.jpg,0
3,train/cat.3406.jpg,0
4,train/cat.4369.jpg,0
5,train/cat.7660.jpg,0
6,train/cat.8553.jpg,0
7,train/cat.9895.jpg,0
8,train/cat.1211.jpg,0
9,train/cat.6218.jpg,0
